# 🌊 Capítulo 06: Streaming e Batches
## Curso: Apache Arrow + DuckDB

Neste capítulo, exploraremos como processar grandes volumes de dados de forma eficiente utilizando streaming e o conceito de batches (lotes) do Apache Arrow integrados ao DuckDB.

### Tópicos abordados:
- **Record Batches**: A unidade fundamental de dados no Arrow.
- **RecordBatchReader**: Interface para leitura de fluxos de dados.
- **Processamento incremental**: Como evitar carregar tudo na memória.
- **Iteradores e geradores**: Integração com fluxos Python.
- **Controle de memória**: Melhores práticas para eficiência.

In [24]:
import sys
import io
import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Configuração para saída correta no Windows (opcional em Notebooks, mas boa prática)
# sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [25]:
# Gerando dados de exemplo globais
print("Gerando dados de exemplo...")
data = pa.table({
    'id': range(1000),
    'valor': np.random.randn(1000),
    'categoria': np.random.choice(['A', 'B', 'C'], 1000)
})
print(f"Tabela PyArrow criada: {data.num_rows} linhas")

# Conexão DuckDB em memória
con = duckdb.connect()
print("Conexão DuckDB estabelecida.")

Gerando dados de exemplo...
Tabela PyArrow criada: 1000 linhas
Conexão DuckDB estabelecida.


## 1️⃣ Record Batches

O `RecordBatch` é uma coleção de arrays de mesmo tamanho. Enquanto uma `Table` pode ser composta por vários pedaços (chunks), um `RecordBatch` é uma estrutura contígua na memória, ideal para processamento em streaming.

### Exemplo Prático:
Vamos converter nossa tabela para uma lista de batches e inspecioná-los.

In [26]:
# Convertendo tabela para batches com tamanho máximo de 200 linhas cada
batches = data.to_batches(max_chunksize=200)
print(f"Total de batches gerados: {len(batches)}")

# Inspecionando o primeiro batch
batch = batches[0]
print(f"Tipo do objeto: {type(batch)}")
print(f"Número de linhas no batch: {batch.num_rows}")

# O DuckDB pode consultar um batch individual como se fosse uma tabela comum
result = con.execute("SELECT count(*), avg(valor) FROM batch").df()
print("\nResultado da consulta DuckDB sobre um RecordBatch:")
print(result)

Total de batches gerados: 5
Tipo do objeto: <class 'pyarrow.lib.RecordBatch'>
Número de linhas no batch: 200

Resultado da consulta DuckDB sobre um RecordBatch:
   count_star()  avg(valor)
0           200   -0.093368


## 2️⃣ RecordBatchReader

O `RecordBatchReader` é um iterador que permite ler batches um por um sem carregar todo o conjunto na memória. Ele é fundamental para fluxos de dados (streaming).

### Exemplo Prático:
Gerando um reader a partir de uma consulta DuckDB.

In [27]:
# Criando um RecordBatchReader a partir de uma consulta no DuckDB
# O parâmetro rows_per_batch controla o tamanho ideal do streaming
reader = con.execute("SELECT * FROM data WHERE valor > 0").fetch_record_batch(rows_per_batch=150)

print(f"Tipo do reader: {type(reader)}")

# Lendo o próximo batch disponível
try:
    batch = reader.read_next_batch()
    print(f"Batch lido: {batch.num_rows} linhas")
except StopIteration:
    print("Fim do stream")

# Podemos ler o restante como uma tabela Arrow completa se necessário
final_table = reader.read_all()
print(f"Linhas restantes lidas de uma vez: {final_table.num_rows}")

Tipo do reader: <class 'pyarrow.lib.RecordBatchReader'>
Batch lido: 150 linhas
Linhas restantes lidas de uma vez: 333


## 3️⃣ Processamento Incremental

Processar dados de forma incremental é a chave para trabalhar com datasets maiores que a memória RAM disponível.

### Exemplo Prático:
Iterando sobre batches para calcular uma soma acumulada sem carregar tudo.

In [28]:
# Refazendo o reader
reader = con.execute("SELECT valor FROM data").fetch_record_batch(rows_per_batch=250)

total_rows = 0
running_sum = 0.0

print("Iniciando processamento incremental...")
for batch in reader:
    # O batch é um RecordBatch. Podemos convertê-lo para pandas ou numpy para processar
    # ou usar funções de computação do próprio Arrow
    chunk_sum = np.sum(batch.column('valor').to_numpy())
    running_sum += chunk_sum
    total_rows += batch.num_rows
    print(f"-> Processado batch de {batch.num_rows} linhas. Soma parcial: {running_sum:.2f}")

print(f"\nProcessamento concluído!")
print(f"Total de linhas vistas: {total_rows}")
print(f"Soma total calculada: {running_sum:.2f}")

# Validação com DuckDB simples
final_sum = con.execute("SELECT sum(valor) FROM data").fetchone()[0]
print(f"Validação DuckDB: {final_sum:.2f}")

Iniciando processamento incremental...
-> Processado batch de 250 linhas. Soma parcial: -13.53
-> Processado batch de 250 linhas. Soma parcial: -11.11
-> Processado batch de 250 linhas. Soma parcial: -37.01
-> Processado batch de 250 linhas. Soma parcial: -44.17

Processamento concluído!
Total de linhas vistas: 1000
Soma total calculada: -44.17
Validação DuckDB: -44.17


## 4️⃣ Iteradores e Geradores

Podemos integrar o Apache Arrow com geradores Python, permitindo que qualquer fluxo de dados seja exposto como um `RecordBatchReader`.

### Exemplo Prático:
Criando um gerador de batches e convertendo-o em um Reader.

In [30]:
def custom_batch_gen(row_count=1000, batch_size=200):
    """Gera RecordBatches de dados aleatórios sob demanda."""
    for i in range(0, row_count, batch_size):
        yield pa.RecordBatch.from_pydict({
            'seq': range(i, min(i + batch_size, row_count)),
            'rand': np.random.rand(min(batch_size, row_count - i))
        })

# Criando o schema necessário para o reader
schema = pa.schema([
    ('seq', pa.int64()),
    ('rand', pa.float64())
])

# Criando o RecordBatchReader a partir do gerador
gen = custom_batch_gen()
reader_from_gen = pa.RecordBatchReader.from_batches(schema, gen)

print(f"Reader criado de gerador: {type(reader_from_gen)}")

# Consumindo via DuckDB (o DuckDB entende RecordBatchReader como fonte)
result = con.execute("SELECT avg(rand) FROM reader_from_gen").df()
print("\nResultado da consulta sobre o gerador:")
print(result)

Reader criado de gerador: <class 'pyarrow.lib.RecordBatchReader'>

Resultado da consulta sobre o gerador:
   avg(rand)
0   0.489407


## 5️⃣ Controle de Memória

Gerenciar o tamanho dos batches e liberar referências é crucial no processamento de grandes volumes de dados.

### Exemplo Prático:
Aferição de tamanho de memória e limpeza de variáveis.

In [31]:
import os
import psutil

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # em MB

print(f"Uso de memória inicial: {get_memory_usage():.2f} MB")

# Tabelas e Batches ocupam memória
print(f"Tamanho estimado da tabela 'data': {data.nbytes / 1024:.2f} KB")

# Para datasets gigantes, evite read_all()
# Prefira processar o iterator e descartar o batch o quanto antes

# Exemplo de limpeza manual se necessário
vars_to_clean = ['data', 'batches', 'final_table']
for var in vars_to_clean:
    if var in locals():
        del locals()[var]

import gc
gc.collect()

print(f"Uso de memória após limpeza: {get_memory_usage():.2f} MB")

Uso de memória inicial: 118.88 MB
Tamanho estimado da tabela 'data': 20.51 KB
Uso de memória após limpeza: 118.90 MB


---
## 🎯 Conclusão do Capítulo 06

Neste capítulo aprendemos:
1. Trabalhar com **RecordBatches** para processamento contíguo.
2. Utilizar o **RecordBatchReader** para ler fluxos de dados sem sobrecarregar a memória.
3. Técnicas de **processamento incremental** integrando DuckDB e Arrow.
4. Como expor geradores Python como leitores Arrow.
5. A importância do controle de memória no mundo de Big Data.

Próximo passo: **Capítulo 07: IPC e Serialização**.